In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd 
import time
import numpy as np

/Users/emmastiefel/Documents/GitHub/Kpop Language Billboard Data Processing/lyrics-env/bin/python


In [2]:
#spotipy documentation: https://spotipy.readthedocs.io/en/2.16.1/
#basic tutorial: https://medium.com/better-programming/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37

##setup spotify api interface
client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
##pull data for baby shark
test_search = sp.search('track:Baby Shark artist:Pinkfong', type='track')

In [5]:
#print song id for test query - just take top return
test_id = test_search['tracks']['items'][0]['id']
print(test_id)
test_name = test_search['tracks']['items'][0]['name']
print(test_name)
test_artist = test_search['tracks']['items'][0]['album']['artists'][0]['name']
print(test_artist)

5ygDXis42ncn6kYG14lEVG
Baby Shark
Pinkfong


In [6]:
#get and print features for test song
test_features = sp.audio_features(test_id)[0]
print('valence (0-1)', test_features['valence'])
print('danceability (0-1)', test_features['danceability'])

valence (0-1) 0.777
danceability (0-1) 0.829


In [28]:
#test out using api with sample query
test_search = sp.search('track:DNA artist:BTS', type='track')

In [52]:
#print song id for test query - just take top return
test_id = test_search['tracks']['items'][0]['id']
print(test_id)
test_name = test_search['tracks']['items'][0]['name']
print(test_name)
test_artist = test_search['tracks']['items'][0]['album']['artists'][0]['name']
print(test_artist)

5SE57ljOIUJ1ybL9U6CuBH
DNA
BTS


In [37]:
#get and print features for test song
test_features = sp.audio_features(test_id)[0]
print('valence (0-1)', test_features['valence'])
print('danceability (0-1)', test_features['danceability'])

valence (0-1) 0.687
danceability (0-1) 0.598


In [87]:
##look up tracks and get features for each bbh100 song
#load data, add columns for audio features
song_data = pd.read_csv('Sentiment_Processed_BBH100_Kpop.csv')
song_data['valence'] = [0.0 for x in song_data['artist']]
song_data['danceability'] = [0.0 for x in song_data['artist']]

#separate out main artist: split by feat. and and
song_data['main_artist'] = song_data['artist'].str.replace(r"(feat.*)", '')
song_data['main_artist'] = song_data['main_artist'].str.replace(r"(and.*)", '')
song_data['main_artist'] = song_data['main_artist'].str.replace(r"(,.*)", '')
song_data['main_artist'] = song_data['main_artist'].str.replace(r"(&.*)", '')

#remove parentheticals from song name and artist name
song_data['search_title'] = song_data['song_title'].str.replace(r"(\(.*\))", "")
song_data['main_artist'] = song_data['main_artist'].str.replace(r"(\(.*\))", "")

In [88]:
for i, r in song_data.iterrows():
    song = r['search_title']
    artist = r['main_artist']
    print(song, artist)

    #search spotify
    spotify_search = sp.search('track:'+song+' artist:'+artist+'', type='track')
    time.sleep(6)

    #if it successfully finds a song, print out validation info
    if len(spotify_search['tracks']['items']) > 0:
        #print song id for test query - just take top return
        this_id = spotify_search['tracks']['items'][0]['id']
        this_name = spotify_search['tracks']['items'][0]['name']
        this_artist = spotify_search['tracks']['items'][0]['album']['artists'][0]['name']
        print(this_name, this_artist)

        this_features = sp.audio_features(this_id)[0]
        song_data.at[i, 'valence'] = this_features['valence']
        song_data.at[i, 'danceability'] = this_features['danceability']

    else:
        print('No results :(')
        song_data.at[i, 'valence'] = np.nan
        song_data.at[i, 'danceability'] = np.nan

    print('___________')

Nobody  Wonder Girls
Nobody Wonder Girls
___________
Gangnam Style Psy
Gangnam Style (강남스타일) PSY
___________
Gentleman Psy
Gentleman Various Artists
___________
Hangover Psy 
Hangover PSY
___________
Daddy Psy 
DADDY PSY
___________
Lifted CL
LIFTED CL
___________
DNA BTS
DNA BTS
___________
MIC Drop  BTS 
MIC Drop (Steve Aoki Remix) [Full Length Edition] BTS
___________
Fake Love BTS
FAKE LOVE BTS
___________
Ddu-Du Ddu-Du Blackpink
DDU-DU DDU-DU BLACKPINK
___________
Idol BTS 
IDOL BTS
___________
Kiss and Make Up Dua Lipa 
Kiss and Make Up Dua Lipa
___________
Waste It on Me Steve Aoki 
Waste It On Me Steve Aoki
___________
Kill This Love Blackpink
Kill This Love BLACKPINK
___________
Boy with Luv BTS 
Boy With Luv (feat. Halsey) BTS
___________
Make It Right  Make It Right remix BTS  BTS 
Make It Right (feat. Lauv) [Acoustic Remix] BTS
___________
Chicken Noodle Soup J-Hope 
Chicken Noodle Soup (feat. Becky G) j-hope
___________
Black Swan BTS
Black Swan BTS
___________
On BTS
Life

In [98]:
##manually fix mistakes

# on by bts
on_url = 'https://open.spotify.com/album/6mJZTV8lCqnwftYZa94bXS?highlight=spotify:track:2QyuXBcV1LJ2rq01KhreMF'
on_features = sp.audio_features(on_url)[0]
on_index = song_data.loc[song_data['search_title'] == 'On'].index[0]
song_data.at[on_index, 'valence'] = on_features['valence']
song_data.at[on_index, 'danceability'] = on_features['danceability']

#daechwita
dae_url = 'https://open.spotify.com/album/0zhGddZ83RpCgnelKWa2qS?highlight=spotify:track:0H4ugk6rhnXmTl47ayy9O5'
dae_features = sp.audio_features(dae_url)[0]
dae_index = song_data.loc[song_data['search_title'] == 'Daechwita'].index[0]
song_data.at[dae_index, 'valence'] = dae_features['valence']
song_data.at[dae_index, 'danceability'] = dae_features['danceability']

In [105]:
#scale valence scores so they match the range (-1, 1) for sentiment polarity scores
song_data['scaled_valence'] = [s*2 - 1 for s in song_data['valence']]
song_data[['valence', 'scaled_valence']]

,valence,scaled_valence
0,0.946,0.892
1,0.749,0.498
2,0.655,0.310
3,0.355,-0.290
4,0.753,0.506
5,0.285,-0.430
6,0.687,0.374
7,0.540,0.080
8,0.344,-0.312
9,0.389,-0.222


In [118]:
##more random processing
# clean hanging space from names
song_data['main_artist'] = song_data['main_artist'].str.strip()
#fix one random name mistake
r_i = song_data.loc[song_data['main_artist'] == 'BTS  BTS'].index[0]
song_data.at[r_i, 'main_artist'] = 'BTS'

IndexError: index 0 is out of bounds for axis 0 with size 0

In [119]:
song_data.to_csv('Spotify_BBH100_Kpop.csv')

In [131]:
### scrape data for wdss songs as well
wdss_data = pd.read_csv('WDSS_Kpop_Translations.csv')
wdss_data['valence'] = [0.0 for x in wdss_data['artist']]
wdss_data['danceability'] = [0.0 for x in wdss_data['artist']]
wdss_data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,chart_date,artist,song_title,original_lyrics,translated_lyrics,peak_position,entry_weeks,Total\nweeks,main_artist,missing_lyrics,missing_trans_lyrics,valence,danceability
0,0,0,3,2010-08-07,Shinee,Lucifer,Hangul\n\n숨을 곳도 찾지 못해 나는\n피하려고 애써 봐도\n거부조차 할 수...,Even if I try to avoid you\nI can't find a pla...,3,20,57,Shinee,False,False,0.0,0.0
1,1,1,5,2010-10-02,2NE1,Can't Nobody,[Minzy] 갈만큼 가겠지\n오늘밤도 길겠지\n분위기 타겠지\n졸졸 따라 올 테지...,NaN,2,3,6,2NE1,False,False,0.0,0.0
2,2,2,8,2010-10-23,Shinee,Hello,Hangul\n\n이럴 때를 보면 나 어리기는 한가봐\n눈 앞에 두고도 어찌 할 줄...,NaN,3,8,17,Shinee,False,False,0.0,0.0
3,3,3,9,2010-10-30,2PM,I'll Be Back,[Junsu]\nI'll be back\nNeon dashi nareul chaje...,NaN,3,3,3,2PM,False,False,0.0,0.0
4,4,4,15,2011-04-16,Jaejoong,"To You It's Goodbye, To Me It's Waiting",A\n\n* About Time by Cosmic Boy ft. Thama & GI...,NaN,2,1,1,Jaejoong,False,False,0.0,0.0


In [132]:
for i, r in wdss_data.iterrows():
    song = r['song_title']
    artist = r['main_artist']
    print(song, artist)

    #search spotify
    spotify_search = sp.search('track:'+song+' artist:'+artist+'', type='track')
    time.sleep(6)

    #if it successfully finds a song, print out validation info
    if len(spotify_search['tracks']['items']) > 0:
        #print song id for test query - just take top return
        this_id = spotify_search['tracks']['items'][0]['id']
        this_name = spotify_search['tracks']['items'][0]['name']
        this_artist = spotify_search['tracks']['items'][0]['album']['artists'][0]['name']
        print(this_name, this_artist)

        this_features = sp.audio_features(this_id)[0]
        wdss_data.at[i, 'valence'] = this_features['valence']
        wdss_data.at[i, 'danceability'] = this_features['danceability']

    else:
        print('No results :(')
        wdss_data.at[i, 'valence'] = np.nan
        wdss_data.at[i, 'danceability'] = np.nan

    print('___________')

Lucifer Shinee
LUCIFER SHINee
___________
Can't Nobody 2NE1
No results :(
___________
Hello Shinee
Hello SHINee
___________
I'll Be Back 2PM
No results :(
___________
To You It's Goodbye, To Me It's Waiting Jaejoong
No results :(
___________
Love Song Big Bang
Love Song BIGBANG
___________
Abandoned feat. Dok2 Jay Park
No results :(
___________
Hands Up 2:00 PM
No results :(
___________
Hate You 2NE1
Hate You 2NE1
___________
Good Bye Baby miss A
Good-bye Baby miss A
___________
Ugly 2NE1
Ugly 2NE1
___________
I Am the Best 2NE1
I Am The Best 2NE1
___________
A-CHa Super Junior
A-CHA SUPER JUNIOR
___________
Be My Baby Wonder Girls
Be My Baby Wonder Girls
___________
You and I IU
You and me Iuno
___________
NalinA Block B
NalinA Block B
___________
Know Your Name feat. Dok2 Jay Park
Know Your Name (feat. Dok2) Jay Park
___________
Blue Big Bang
Blue BIGBANG
___________
Bad Boy Big Bang
BAD BOY BIGBANG
___________
Fantastic Baby Big Bang
Fantastic Baby BIGBANG
___________
Monster Big Ba

In [133]:
#scale valence scores so they match the range (-1, 1) for sentiment polarity scores
wdss_data['scaled_valence'] = [s*2 - 1 for s in wdss_data['valence']]
wdss_data[['valence', 'scaled_valence']]

,valence,scaled_valence
0,0.866,0.732
1,NaN,NaN
2,0.626,0.252
3,NaN,NaN
4,NaN,NaN
...,...,...
240,NaN,NaN
241,0.901,0.802
242,0.450,-0.100
243,0.364,-0.272


In [134]:
wdss_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,chart_date,artist,song_title,original_lyrics,translated_lyrics,peak_position,entry_weeks,Total\nweeks,main_artist,missing_lyrics,missing_trans_lyrics,valence,danceability,scaled_valence
0,0,0,3,2010-08-07,Shinee,Lucifer,Hangul\n\n숨을 곳도 찾지 못해 나는\n피하려고 애써 봐도\n거부조차 할 수...,Even if I try to avoid you\nI can't find a pla...,3,20,57,Shinee,False,False,0.866,0.750,0.732
1,1,1,5,2010-10-02,2NE1,Can't Nobody,[Minzy] 갈만큼 가겠지\n오늘밤도 길겠지\n분위기 타겠지\n졸졸 따라 올 테지...,NaN,2,3,6,2NE1,False,False,NaN,NaN,NaN
2,2,2,8,2010-10-23,Shinee,Hello,Hangul\n\n이럴 때를 보면 나 어리기는 한가봐\n눈 앞에 두고도 어찌 할 줄...,NaN,3,8,17,Shinee,False,False,0.626,0.761,0.252
3,3,3,9,2010-10-30,2PM,I'll Be Back,[Junsu]\nI'll be back\nNeon dashi nareul chaje...,NaN,3,3,3,2PM,False,False,NaN,NaN,NaN
4,4,4,15,2011-04-16,Jaejoong,"To You It's Goodbye, To Me It's Waiting",A\n\n* About Time by Cosmic Boy ft. Thama & GI...,NaN,2,1,1,Jaejoong,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,240,240,1387,2020-11-07,"K/DA (G)I-dle, Madison Beer, Jaira Burns, & Le...",More,NaN,NaN,1,5,5,"K/DA (G)I-dle, Madison Beer, Jaira Burns, & Le...",True,False,NaN,NaN,NaN
241,241,241,1393,2020-11-14,CL,Hwa,"[Chorus]\nYeah, you know\nYou want some tight ...",Nana Nanna Nana Nana~ (*3)\nThe rose of sharon...,3,1,1,CL,False,False,0.901,0.665,0.802
242,242,242,1405,2020-12-05,BTS,Life Goes On,"[방탄소년단 ""Life Goes On"" 가사]\n\n[Verse 1: Jung Ko...","[Intro]\nI remember\nI, I, I remember\nAh, ah\...",1,1,1,BTS,False,False,0.450,0.566,-0.100
243,243,243,1406,2020-12-05,BTS,Blue & Grey,"[방탄소년단 ""Blue & Grey"" 가사]\n\n[Intro: V, Jung Ko...","[Intro: V, Jung Kook]\nWhere is my angel?\nThe...",2,1,1,BTS,False,False,0.364,0.547,-0.272


In [135]:
wdss_data.to_csv('Spotify_WDSS_Kpop.csv')